In [9]:
!pip install mxnet opencv-python boto3 torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 42.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 382.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 30.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 48.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 620.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 26.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━

In [12]:
import os
import boto3
import sagemaker
import numpy as np
import mxnet as mx
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.inputs import TrainingInput
from mxnet.gluon.data.vision import datasets, transforms
from mxnet import recordio
from PIL import Image
from torchvision import datasets, transforms
import io

In [ ]:
# Set up the environment
role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [ ]:
def fetch_and_convert_to_recordio():
    training_dataset = datasets.CIFAR10(root='data', train=True, transform=transforms.ToTensor(), download=True)
    validation_dataset = datasets.CIFAR10(root='data', train=False, transform=transforms.ToTensor())
    
    os.makedirs('data/train', exist_ok=True)
    os.makedirs('data/validation', exist_ok=True)
    
    # Convert Training Dataset to RecordIO
    record = recordio.MXRecordIO('data/train/train.rec', 'w')
    for i in range(len(training_dataset)):
        data, label = training_dataset[i]
        data = (data.numpy() * 255).astype(np.uint8).transpose(1, 2, 0)  # Convert to numpy array
        header = recordio.IRHeader(0, label, i, 0)
        s = recordio.pack_img(header, data, quality=100)
        record.write(s)
    record.close()
    
    # Convert Validation Dataset to RecordIO
    record = recordio.MXRecordIO('data/validation/validation.rec', 'w')
    for i in range(len(validation_dataset)):
        data, label = validation_dataset[i]
        data = (data.numpy() * 255).astype(np.uint8).transpose(1, 2, 0)  # Convert to numpy array
        header = recordio.IRHeader(0, label, i, 0)
        s = recordio.pack_img(header, data, quality=100)
        record.write(s)
    record.close()

In [ ]:
fetch_and_convert_to_recordio()

In [ ]:
# Upload the data to S3
training_data = sess.upload_data(path='data/train', key_prefix='data/cifar10/train')
validation_data = sess.upload_data(path='data/validation', key_prefix='data/cifar10/validation')

In [ ]:
# Train model using SageMaker
training_image = get_image_uri(region, 'image-classification', repo_version="latest")

In [ ]:
image_classifier = sagemaker.estimator.Estimator(training_image,
                                                 role,
                                                 instance_count=1,
                                                 instance_type='ml.p2.xlarge',
                                                 sagemaker_session=sess)

In [ ]:
image_classifier.set_hyperparameters(num_layers=18,
                                     image_shape='3,32,32',
                                     num_classes=10,
                                     num_training_samples=50000,
                                     epochs=10)

In [ ]:
train_data = TrainingInput(training_data, distribution='FullyReplicated', 
                           content_type='application/x-recordio', s3_data_type='S3Prefix')
validation_data = TrainingInput(validation_data, distribution='FullyReplicated', 
                                 content_type='application/x-recordio', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}

In [ ]:
image_classifier.fit(inputs=data_channels, logs=True)

In [ ]:
# Deploy the model
predictor = image_classifier.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1)

In [17]:
def preprocess_input(image_path):
    # Open image and resize to 32x32 pixels
    img = Image.open(image_path)
    img = img.resize((32, 32))
    
    # Convert image to numpy array and normalize
    img_array = np.array(img)
    img_array = img_array / 255.0
    
    # Reshape image array and expand dimensions
    img_array = img_array.reshape(1, 3, 32, 32)
    
    # Convert numpy array to bytes
    img_bytes = img_array.tobytes()
    
    return img_bytes

In [38]:
import cv2
import boto3

# Load image
img = cv2.imread('data/images/dog.jpg')
img = cv2.resize(img, (32, 32)) # Resize the image to the expected size

# Convert image to byte array
_, img_encoded = cv2.imencode('.jpg', img)
img_bytes = img_encoded.tostring()

# Use boto3 to send request
client = boto3.client('runtime.sagemaker')
response = client.invoke_endpoint(
    EndpointName='image-classification-2023-08-31-13-46-20-134',
    Body=img_bytes,
    ContentType='application/x-image',
)

/tmp/ipykernel_11674/584243589.py:10: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  img_bytes = img_encoded.tostring()


In [39]:
import boto3
import ast
import numpy as np

# Define the CIFAR-10 classes
classes = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

# Assume response is the response from the SageMaker endpoint
response_body = response['Body'].read()
print(response_body)
response_string = response_body.decode('utf-8')
probabilities = ast.literal_eval(response_string)

# Get the index with the maximum probability
predicted_class_index = np.argmax(probabilities)

# Get the name of the predicted class
predicted_class_name = classes[predicted_class_index]

# Print the predicted class
print(f"The model predicts the image belongs to class: {predicted_class_name} ({predicted_class_index})")


b'[0.000829699682071805, 0.00012848513142671436, 0.0399332121014595, 0.030882712453603745, 0.048324260860681534, 0.3547470271587372, 0.46455663442611694, 0.004775063134729862, 0.05568608641624451, 0.00013684295117855072]'
The model predicts the image belongs to class: Frog (6)
